# Climate-Friendly Food Systems (CFFS) Labelling Project

### The University of British Columbia

***

## Part III: Update Information and Mapping

## Set up and Import Libraries

In [1]:
#pip install -r requirements.txt

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import csv
from itertools import islice
from decimal import Decimal
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse
import openpyxl
import pytest
from datetime import datetime

In [2]:
# RUN ONLY ONCE
# os.chdir is used to change the current directory to the specified path
os.chdir("../") # Sets path to the repo folder as it is one level above where this file exists!
path = os.getcwd()
print(path)

/Users/vivaanwadhwa/Documents/GitHub/CFFS_sharon_2024


***
## Import Preprocessed Datasets

In [3]:
# Creates a DataFrame from reading a csv file 
Preps = pd.read_csv(os.path.join(os.getcwd(), "data", "cleaning", "Preps_Unit_Cleaned.csv"))
Preps.head()

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-68207,24|BATCH|Poblano Pork Stew*,100.0,PTN,PREP,100.0,PTN
1,P-61322,AIOLI|Pesto,4.0,L,ZDONT USE OK - PREP,4000.0,ml
2,P-46066,Almond Milk Yeild,900.0,ml,NaN,900.0,ml
3,P-8814,Aoli- RoastedGarlic Thyme,1.1,L,NaN,1100.0,ml
4,P-49605,APPLE|Cubed,75.0,g,NaN,75.0,g


In [4]:
# Creates a DataFrame from reading a csv file 
ghge_factors = pd.read_csv(os.path.join(os.getcwd(), "data", "external", "ghge_factors.csv"))
ghge_factors.head()

,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,1,beef & buffalo meat,41.3463
1,2,lamb/mutton & goat meat,41.6211
2,3,pork (pig meat),9.8315
3,4,"poultry (chicken, turkey)",4.3996
4,5,butter,11.4316


In [5]:
# Creates a DataFrame from reading a csv file 
nitro_factors = pd.read_csv(os.path.join(os.getcwd(), "data", "external", "nitrogen_factors.csv"))
nitro_factors.head()

,Category ID,Food Category,g N lost/kg product
0,1,beef & buffalo meat,329.50
1,2,lamb/mutton & goat meat,231.15
2,3,pork (pig meat),132.80
3,4,"poultry (chicken, turkey)",116.80
4,5,butter,100.35


In [6]:
# Creates a DataFrame from reading a csv file 
water_factors = pd.read_csv(os.path.join(os.getcwd(), "data", "external", "water_factors.csv"))
water_factors.head()

,Category ID,Food Category,Freshwater Withdrawals (L/FU),Stress-Weighted Water Use (L/FU)
0,1,beef & buffalo meat,1677.200,61309.000
1,2,lamb/mutton & goat meat,461.200,258.900
2,3,pork (pig meat),1810.300,54242.700
3,4,"poultry (chicken, turkey)",370.300,333.500
4,5,butter,1010.176,50055.168


In [7]:
# Creates a DataFrame from reading a csv file 
land_factors = pd.read_csv(os.path.join(os.getcwd(), "data", "external", "land_factors.csv"))
land_factors.rename(columns={'km^2 land use/kg product': 'Land Use (m^2)'}, inplace=True)
land_factors['Land Use (m^2)'] *= 1000
land_factors.head()

,Category ID,Food Category,Land Use (m^2)
0,1,beef & buffalo meat,0.12645
1,2,lamb/mutton & goat meat,0.14320
2,3,pork (pig meat),0.02102
3,4,"poultry (chicken, turkey)",0.01151
4,5,butter,0.01395


In [8]:
# Load current Items List with assigned Emission Factors Category ID
Items_Assigned = pd.read_csv(os.path.join(os.getcwd(), "data", "mapping", "Items_List_Assigned.csv"))
Items_Assigned.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,NaN
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,NaN
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,NaN
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,NaN
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,NaN


In [10]:
new_items = pd.read_csv("data/mapping/new items added/New_Items_2024/New_Items+Added_2024-08-09.csv")
new_items

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-3310,54,BASIL LEAF*LG SKR,1.00,LG SKR,775.000,g,SPICES
1,I-72216,1,BEEF STEW MEAT TRNB,1.00,lb,1.000,lb,MEAT
2,I-72465,21,CEREAL GRANOLA*PLAIN,3.00,bag,1.000,Kg,FOOD - GROCERY
3,I-4009,4,CHICK THIGH B/LS/L DICED,1.00,Kg,1.000,Kg,POULTRY
4,I-13634,12,COD GREY 3oz LOIN IQF MSC,1.00,lb,1.000,lb,SEAFOOD
5,I-70559,53,COFFEE COLD BREW,2.00,JUG,1.000,JUG,BEVERAGE
6,I-65800,8,CREAM 18%*946ML,1.00,ea,946.000,ml,DAIRY
7,I-71938,41,FRIES 3/8IN*CRINKLE,6.00,bag,5.000,lb,PRODUCE
8,I-73346,41,FRIES SPIRAL*REDSTONE,6.00,bag,4.000,lb,PRODUCE
9,I-71889,41,FRIES WAFFLE*SPICY CRUNCH,6.00,bag,4.000,lb,PRODUCE


In [11]:
item_nonstd = pd.read_csv("data/cleaning/Items_Nonstd.csv")
item_nonstd

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe,Description
0,I-1273,1.000,LOAF,1.0,0.4409,P-13648,SOURDOUGH BREAD COUNTRY
1,I-17151,5.000,CT,1.0,0.5000,P-28369,CIABATTA TUSCANO SLICED
2,I-4766,1.000,CT,1.0,0.0017,P-43206,ORANGES SUNKISTUS 88
3,I-1852,1.000,CT,1.0,0.0011,P-46066,ALMOND BEV - ORIGINAL
4,I-7055,1.000,SHEET,1.0,0.0313,P-50725,FOCACCIA
5,I-24229,1.000,LOAF,1.0,0.0625,P-51992,LOAF ORGANIC PAN SLD CHEEKY
6,I-3103,1.000,CT,1.0,1.0000,P-55079,SPRING ROLL WRAP 8.5 FROZEN
7,I-1033,1.000,CT,1.0,0.1667,P-57968,PITA GREEK*THK 7IN
8,I-65852,1.000,each,1.0,0.1667,P-57968,"CIABATTA WH/GRAIN 6"""
9,I-13420,1.000,LOAF,1.0,0.0833,P-62067,LOAF GF WHITE UDI'S SANDWICH


In [12]:
preps_nonstd = pd.read_csv("data/cleaning/Preps_NonstdUom.csv")
preps_nonstd

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom


In [13]:
conv_updatecov = pd.read_csv("data/cleaning/update/Conv_UpdateConv.csv")
conv_updatecov

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,I-67659,0.008818,1.0,each,113.398,g
1,I-28697,0.005181,1.0,ea,193.000,g
2,I-47441,0.005181,1.0,ea,193.000,g
3,I-1905,0.012500,1.0,CT,80.000,g
4,I-47440,0.005051,1.0,ea,198.000,g
...,...,...,...,...,...,...
599,I-63874,0.022222,1.0,CT,45.000,g
600,I-8856,0.035714,1.5,ea,42.000,g
601,I-1196,0.017544,0.5,CT,28.500,g
602,I-10605,0.012821,1.0,CT,78.000,g


In [14]:
conv_updatecov.loc[conv_updatecov["ConversionId"] == "P-54581"]

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom


In [15]:
Preps.loc[Preps["PrepId"] == "P-54581"]

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom


# Update Conversion_Added.csv

In [16]:
conversions_added = pd.read_csv(os.path.join(os.getcwd(), "data", "cleaning", "Conversions_Added.csv"))
conversions_added

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,NaN,1.000000,1.000,XXX,1.00,L
1,NaN,0.877193,1.000,1.14L,1.14,L
2,NaN,0.666667,1.000,1.5L,1.50,L
3,NaN,0.571429,1.000,1.75 L,1.75,L
4,NaN,0.500000,1.000,2L,2.00,L
...,...,...,...,...,...,...
660,I-72414,0.000357,0.025,cs,70.00,g
661,I-62928,0.010526,1.000,ea,95.00,g
662,I-73417,0.007143,1.000,ea,140.00,g
663,I-73414,0.007143,1.000,ea,140.00,g


In [17]:
# Concatenate the DataFrames
combined_df = pd.concat([conv_updatecov, conversions_added], ignore_index=True)
combined_df

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,I-67659,0.008818,1.000,each,113.398,g
1,I-28697,0.005181,1.000,ea,193.000,g
2,I-47441,0.005181,1.000,ea,193.000,g
3,I-1905,0.012500,1.000,CT,80.000,g
4,I-47440,0.005051,1.000,ea,198.000,g
...,...,...,...,...,...,...
1264,I-72414,0.000357,0.025,cs,70.000,g
1265,I-62928,0.010526,1.000,ea,95.000,g
1266,I-73417,0.007143,1.000,ea,140.000,g
1267,I-73414,0.007143,1.000,ea,140.000,g


In [18]:
# Add the conv_updatecov file to the masterfile, Conversions_Added
path = os.path.join(os.getcwd(), "data", "cleaning", "Conversions_Added.csv")
combined_df.to_csv(path, index = False, header = True)

***
## Import Update Info

In [19]:
# Import list of prep that need convert uom to standard uom manually
Manual_PrepU = pd.read_csv(os.path.join(os.getcwd(), "data", "cleaning", "update", "Preps_UpdateUom.csv"))
# Here we can see that UOM examples are: each, ea, slice
Manual_PrepU.head()

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-54697,LEMON|Wedge 1/8,8.0,each,PREP,84.0,g
1,P-35132,MARINATED|Lemon & Herb Chx,185.0,ea,PREP,24050.0,g
2,P-51992,YIELD|Bread|Sourdough 5/8,36.0,slice,NaN,1620.0,g
3,P-26234,BATCH|Roasted Garlic Bread,16.0,ea,PREP,1280.0,g
4,P-26170,GRILLED|NaanBread,1.0,ea,PREP,125.0,g


In [20]:
# The row of the DatFrame that contains the PrepId == P-54581
Manual_PrepU.loc[Manual_PrepU["PrepId"] == "P-54581"]

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
212,P-54581,SLICE|Multigrain Bread,22.0,slice,NaN,38.0,g


In [23]:
# Select the file path for new items list with category id
New_Items_Added = pd.read_csv("data/mapping/new items added/New_Items_2024/New_Items+Added_2024-08-09.csv")
New_Items_Added.tail(15)

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
13,I-72745,2,MEAT GYRO*HALAL RAW,4.00,bag,4540.000,g,MEAT
14,I-71922,2,MEATBALL BEEF*CHICK HALAL,2.00,bag,5.000,lb,MEAT
15,I-52698,53,OAT MILK LATTE - VANILLA,6.00,each,1.000,each,BEVERAGE
16,I-72437,24,PANKO CRUMB*BREAD UNTOASTED,11.30,Kg,1.000,Kg,FOOD - GROCERY
17,I-3420,24,PASTA SCOOBI*DOO,9.07,Kg,1.000,Kg,FOOD - GROCERY
18,I-3445,54,PEPPER BLK*DISP SHAKER,48.00,CT,1.000,CT,SPICES
19,I-62913,24,PITA 5IN*THICK T/S,10.00,bag,12.000,ea,BREAD
20,I-61511,3,PORK BELLY RIND ON FZ,1.00,lb,0.454,Kg,MEAT
21,I-72217,3,PORK BUTT BNLS SNLESS DICED FR,1.00,lb,1.000,lb,MEAT
22,I-72215,3,PORK LOIN BONEIN 8OZ FRVP,1.00,lb,1.000,lb,MEAT


In [24]:
# Import list of items that adjusted GHGe factor manually
# Manual_Factor = pd.read_csv(os.path.join(os.getcwd(), "data", "mapping", "Manual_Adjust_Factors.csv"))
# Manual_Factor['Land Use (m^2)'] = 0
# Manual_Factor.head()

### Update Correct Uom for Preps

In [25]:
# Update prep list with manually adjusted uom
for index, row in Manual_PrepU.iterrows():
    PrepId = Manual_PrepU.loc[index, 'PrepId']
    qty = Manual_PrepU.loc[index, 'StdQty']
    uom = Manual_PrepU.loc[index, 'StdUom']
    Preps.loc[Preps['PrepId'] == PrepId, 'StdQty'] = qty
    Preps.loc[Preps['PrepId'] == PrepId, 'StdUom'] = uom

In [26]:
Preps.drop_duplicates(subset=['PrepId'], inplace=True,)

In [27]:
Preps.head()

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-68207,24|BATCH|Poblano Pork Stew*,100.0,PTN,PREP,28800.0,g
1,P-61322,AIOLI|Pesto,4.0,L,ZDONT USE OK - PREP,4000.0,ml
2,P-46066,Almond Milk Yeild,900.0,ml,NaN,900.0,ml
3,P-8814,Aoli- RoastedGarlic Thyme,1.1,L,NaN,1100.0,ml
4,P-49605,APPLE|Cubed,75.0,g,NaN,75.0,g


In [28]:
Preps.shape

(890, 7)

In [29]:
path = os.path.join(os.getcwd(), "data", "cleaning", "Preps_List_Cleaned.csv")
Preps.to_csv(path, index = False, header = True)

In [30]:
Items_Assigned.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,NaN
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,NaN
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,NaN
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,NaN
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,NaN


In [31]:
New_Items_Added.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-3310,54,BASIL LEAF*LG SKR,1.0,LG SKR,775.0,g,SPICES
1,I-72216,1,BEEF STEW MEAT TRNB,1.0,lb,1.0,lb,MEAT
2,I-72465,21,CEREAL GRANOLA*PLAIN,3.0,bag,1.0,Kg,FOOD - GROCERY
3,I-4009,4,CHICK THIGH B/LS/L DICED,1.0,Kg,1.0,Kg,POULTRY
4,I-13634,12,COD GREY 3oz LOIN IQF MSC,1.0,lb,1.0,lb,SEAFOOD


### Import List of New Items with Emission Factors Category ID Assigned

In [32]:
frames = [Items_Assigned, New_Items_Added]
Items_Assigned_Updated = pd.concat(frames).reset_index(drop=True, inplace=False).drop_duplicates()
Items_Assigned_Updated.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,NaN
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,NaN
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,NaN
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,NaN
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,NaN


In [33]:
Items_Assigned_Updated

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,NaN
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,NaN
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,NaN
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,NaN
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,NaN
...,...,...,...,...,...,...,...,...,...
2906,I-36775,25.0,RICE CALROSE BAG,1.00,ea,20.0,Kg,FOOD - GROCERY,NaN
2907,I-71940,25.0,RICE PARBOILED*,20.00,Kg,1.0,Kg,FOOD - GROCERY,NaN
2908,I-72190,58.0,SAUCE MIRIN*SWEET,12.00,bottle,1.0,L,FOOD - GROCERY,NaN
2909,I-10481,26.0,SESAME BLK*,1.00,Kg,2.2,lb,FOOD - GROCERY,NaN


In [34]:
Items_Assigned_Updated.shape

(2911, 9)

In [35]:
# Double brackets used to specify the column as a dataframe and not a series
# converting the 'CategoryID' column in the Items_Assigned_Updated dataframe to numeric data type using the pd.to_numeric()
# use the apply column to apply the coversion to the entire column
Items_Assigned_Updated[['CategoryID']] = Items_Assigned_Updated[['CategoryID']].apply(pd.to_numeric)

In [36]:
Items_Assigned_Updated

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,NaN
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,NaN
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,NaN
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,NaN
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,NaN
...,...,...,...,...,...,...,...,...,...
2906,I-36775,25.0,RICE CALROSE BAG,1.00,ea,20.0,Kg,FOOD - GROCERY,NaN
2907,I-71940,25.0,RICE PARBOILED*,20.00,Kg,1.0,Kg,FOOD - GROCERY,NaN
2908,I-72190,58.0,SAUCE MIRIN*SWEET,12.00,bottle,1.0,L,FOOD - GROCERY,NaN
2909,I-10481,26.0,SESAME BLK*,1.00,Kg,2.2,lb,FOOD - GROCERY,NaN


In [37]:
path = os.path.join(os.getcwd(), "data", "mapping", "Items_List_Assigned.csv")
Items_Assigned_Updated.to_csv(path, index = False, header = True)

## Mapping Items to Footprint Factors

In [38]:
Items_Assigned_Updated

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,NaN
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,NaN
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,NaN
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,NaN
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,NaN
...,...,...,...,...,...,...,...,...,...
2906,I-36775,25.0,RICE CALROSE BAG,1.00,ea,20.0,Kg,FOOD - GROCERY,NaN
2907,I-71940,25.0,RICE PARBOILED*,20.00,Kg,1.0,Kg,FOOD - GROCERY,NaN
2908,I-72190,58.0,SAUCE MIRIN*SWEET,12.00,bottle,1.0,L,FOOD - GROCERY,NaN
2909,I-10481,26.0,SESAME BLK*,1.00,Kg,2.2,lb,FOOD - GROCERY,NaN


In [39]:
# DEFINITION OF MAPPING: assigning certain attributes to data points based on criteria or predefined rules.

# Map GHG footprint factors
# merges two data frames Items_Assigned_Updated and ghge_factors on the columns CategoryID and Category ID, respectively. The 
# resulting data frame is stored in mapping.

# how='left' specifies that a left join is done between Items_Assigned_Updated and ghge_factors.
# left_on='CategoryID' specifies that the join condition for Items_Assigned_Updated should be based on the 'CategoryID' column.
# right_on='Category ID' specifies that the join condition for ghge_factors should be based on the 'Category ID' column.

mapping = pd.merge(Items_Assigned_Updated, ghge_factors.loc[:,['Category ID','Food Category','Active Total Supply Chain Emissions (kg CO2 / kg food)']], 
                  how = 'left',
                  left_on = 'CategoryID', 
                  right_on = 'Category ID')
# if row in mapping has CategoryId == NaN then it assigns the value of the column Active Total Supply Chain Emissions (kg CO2 / kg food)
# to zero
for index in mapping.index:
    if np.isnan(mapping.loc[index,'Category ID']):
        mapping.loc[index,'Active Total Supply Chain Emissions (kg CO2 / kg food)'] = 0
# drops the columns Category ID and Food Category
display(mapping)
mapping = mapping.drop(columns=['Category ID', 'Food Category_x'])
mapping

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category_x,Category ID,Food Category_y,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,NaN,1.0,beef & buffalo meat,41.3463
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,NaN,1.0,beef & buffalo meat,41.3463
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,NaN,1.0,beef & buffalo meat,41.3463
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,NaN,1.0,beef & buffalo meat,41.3463
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,NaN,1.0,beef & buffalo meat,41.3463
...,...,...,...,...,...,...,...,...,...,...,...,...
2906,I-36775,25.0,RICE CALROSE BAG,1.00,ea,20.0,Kg,FOOD - GROCERY,NaN,25.0,rice,2.5345
2907,I-71940,25.0,RICE PARBOILED*,20.00,Kg,1.0,Kg,FOOD - GROCERY,NaN,25.0,rice,2.5345
2908,I-72190,58.0,SAUCE MIRIN*SWEET,12.00,bottle,1.0,L,FOOD - GROCERY,NaN,58.0,sauces & paste,0.0000
2909,I-10481,26.0,SESAME BLK*,1.00,Kg,2.2,lb,FOOD - GROCERY,NaN,26.0,tree nuts and seeds,4.2854


,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category_y,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,beef & buffalo meat,41.3463
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,beef & buffalo meat,41.3463
...,...,...,...,...,...,...,...,...,...,...
2906,I-36775,25.0,RICE CALROSE BAG,1.00,ea,20.0,Kg,FOOD - GROCERY,rice,2.5345
2907,I-71940,25.0,RICE PARBOILED*,20.00,Kg,1.0,Kg,FOOD - GROCERY,rice,2.5345
2908,I-72190,58.0,SAUCE MIRIN*SWEET,12.00,bottle,1.0,L,FOOD - GROCERY,sauces & paste,0.0000
2909,I-10481,26.0,SESAME BLK*,1.00,Kg,2.2,lb,FOOD - GROCERY,tree nuts and seeds,4.2854


In [40]:
# Map nitrogen footprint factors
mapping = pd.merge(mapping, nitro_factors.loc[:,['Category ID','Food Category','g N lost/kg product']], 
                  how = 'left',
                  left_on = 'CategoryID', 
                  right_on = 'Category ID')

for index in mapping.index:
    if np.isnan(mapping.loc[index,'Category ID']):
        mapping.loc[index,'g N lost/kg product'] = 0

mapping = mapping.drop(columns=['Category ID', 'Food Category'])
mapping

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category_y,Active Total Supply Chain Emissions (kg CO2 / kg food),g N lost/kg product
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463,329.50
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463,329.50
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463,329.50
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,beef & buffalo meat,41.3463,329.50
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,beef & buffalo meat,41.3463,329.50
...,...,...,...,...,...,...,...,...,...,...,...
2906,I-36775,25.0,RICE CALROSE BAG,1.00,ea,20.0,Kg,FOOD - GROCERY,rice,2.5345,5.30
2907,I-71940,25.0,RICE PARBOILED*,20.00,Kg,1.0,Kg,FOOD - GROCERY,rice,2.5345,5.30
2908,I-72190,58.0,SAUCE MIRIN*SWEET,12.00,bottle,1.0,L,FOOD - GROCERY,sauces & paste,0.0000,6.75
2909,I-10481,26.0,SESAME BLK*,1.00,Kg,2.2,lb,FOOD - GROCERY,tree nuts and seeds,4.2854,12.20


In [41]:
# Map land footprint factors
mapping = pd.merge(mapping, land_factors.loc[:,['Category ID','Food Category','Land Use (m^2)']], 
                  how = 'left',
                  left_on = 'CategoryID', 
                  right_on = 'Category ID')

for index in mapping.index:
    if np.isnan(mapping.loc[index,'Category ID']):
        mapping.loc[index,'Land Use (m^2)'] = 0

mapping = mapping.drop(columns=['Category ID', 'Food Category'])
mapping

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category_y,Active Total Supply Chain Emissions (kg CO2 / kg food),g N lost/kg product,Land Use (m^2)
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463,329.50,0.12645
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463,329.50,0.12645
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463,329.50,0.12645
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,beef & buffalo meat,41.3463,329.50,0.12645
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,beef & buffalo meat,41.3463,329.50,0.12645
...,...,...,...,...,...,...,...,...,...,...,...,...
2906,I-36775,25.0,RICE CALROSE BAG,1.00,ea,20.0,Kg,FOOD - GROCERY,rice,2.5345,5.30,0.00217
2907,I-71940,25.0,RICE PARBOILED*,20.00,Kg,1.0,Kg,FOOD - GROCERY,rice,2.5345,5.30,0.00217
2908,I-72190,58.0,SAUCE MIRIN*SWEET,12.00,bottle,1.0,L,FOOD - GROCERY,sauces & paste,0.0000,6.75,0.00000
2909,I-10481,26.0,SESAME BLK*,1.00,Kg,2.2,lb,FOOD - GROCERY,tree nuts and seeds,4.2854,12.20,0.00692


In [42]:
# Map water footprint factors
# mapping: DataFrame that is used to assign sustainability-related factors (greenhouse gas emissions, nitrogen loss, and 
# water footprint) to food items based on their category and other attributes. 
mapping = pd.merge(mapping, water_factors.loc[:,['Category ID','Food Category','Freshwater Withdrawals (L/FU)', 'Stress-Weighted Water Use (L/FU)']], 
                  how = 'left',
                  left_on = 'CategoryID', 
                  right_on = 'Category ID')

for index in mapping.index:
    if np.isnan(mapping.loc[index,'Category ID']):
        mapping.loc[index,'Freshwater Withdrawals (L/FU)'] = 0
        mapping.loc[index,'Stress-Weighted Water Use (L/FU)'] = 0

mapping = mapping.drop(columns=['Category ID', 'Food Category'])
mapping.drop_duplicates(subset=["ItemId"], inplace=True)
mapping

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category_y,Active Total Supply Chain Emissions (kg CO2 / kg food),g N lost/kg product,Land Use (m^2),Freshwater Withdrawals (L/FU),Stress-Weighted Water Use (L/FU)
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463,329.50,0.12645,1677.200,61309.000
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463,329.50,0.12645,1677.200,61309.000
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463,329.50,0.12645,1677.200,61309.000
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,beef & buffalo meat,41.3463,329.50,0.12645,1677.200,61309.000
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,beef & buffalo meat,41.3463,329.50,0.12645,1677.200,61309.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2906,I-36775,25.0,RICE CALROSE BAG,1.00,ea,20.0,Kg,FOOD - GROCERY,rice,2.5345,5.30,0.00217,1574.900,4625.600
2907,I-71940,25.0,RICE PARBOILED*,20.00,Kg,1.0,Kg,FOOD - GROCERY,rice,2.5345,5.30,0.00217,1574.900,4625.600
2908,I-72190,58.0,SAUCE MIRIN*SWEET,12.00,bottle,1.0,L,FOOD - GROCERY,sauces & paste,0.0000,6.75,0.00000,20.225,1134.925
2909,I-10481,26.0,SESAME BLK*,1.00,Kg,2.2,lb,FOOD - GROCERY,tree nuts and seeds,4.2854,12.20,0.00692,1823.300,129364.300


In [43]:
mapping[mapping["ItemId"] == "I-4524"]

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category_y,Active Total Supply Chain Emissions (kg CO2 / kg food),g N lost/kg product,Land Use (m^2),Freshwater Withdrawals (L/FU),Stress-Weighted Water Use (L/FU)
2006,I-4524,38.0,BEETS GOLDBC,5.0,lb,1.0,lb,PRODUCE,root vegetables,0.3062,7.9,0.00032,9.9,37.9


In [44]:
mapping["CategoryID"].isnull().sum()

0

### Manully Adjust Footprint Factor for Specific Items

In [45]:
# For Manual_Factor: dataframe with items that adjusted GHGe factor manually.
# It takes the id for Manual_Factor for the index being iterated and if it is equal to item id of mapping dataframe then it sets
# the values of the columns to the manually adjusted values
# Note: the values for the columns in mapping DataFrame is adjusted and not Manual_Factor DataFrame
for index, row in Manual_Factor.iterrows():
    itemId = Manual_Factor.loc[index, 'ItemId']
    ghge = Manual_Factor.loc[index, 'Active Total Supply Chain Emissions (kg CO2 / kg food)']
    nitro = Manual_Factor.loc[index, 'g N lost/kg product']
    water = Manual_Factor.loc[index, 'Freshwater Withdrawals (L/FU)']
    land = Manual_Factor.loc[index, 'Land Use (m^2)']
    str_water = Manual_Factor.loc[index, 'Stress-Weighted Water Use (L/FU)']
    mapping.loc[mapping['ItemId'] == itemId, 'Active Total Supply Chain Emissions (kg CO2 / kg food)'] = ghge
    mapping.loc[mapping['ItemId'] == itemId, 'g N lost/kg product'] = nitro
    mapping.loc[mapping['ItemId'] == itemId, 'Freshwater Withdrawals (L/FU)'] = water
    mapping.loc[mapping['ItemId'] == itemId, 'Stress-Weighted Water Use (L/FU)'] = str_water
    mapping.loc[mapping['ItemId'] == itemId, 'Land Use (m^2)'] = land

NameError: name 'Manual_Factor' is not defined

In [46]:
mapping.drop_duplicates(subset = ['ItemId'], inplace=True)
mapping.dtypes

ItemId                                                     object
CategoryID                                                float64
Description                                                object
CaseQty                                                   float64
CaseUOM                                                    object
PakQty                                                    float64
PakUOM                                                     object
InventoryGroup                                             object
Food Category_y                                            object
Active Total Supply Chain Emissions (kg CO2 / kg food)    float64
g N lost/kg product                                       float64
Land Use (m^2)                                            float64
Freshwater Withdrawals (L/FU)                             float64
Stress-Weighted Water Use (L/FU)                          float64
dtype: object

In [47]:
mapping.shape

(2911, 14)

In [48]:
mapping

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category_y,Active Total Supply Chain Emissions (kg CO2 / kg food),g N lost/kg product,Land Use (m^2),Freshwater Withdrawals (L/FU),Stress-Weighted Water Use (L/FU)
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463,329.50,0.12645,1677.200,61309.000
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463,329.50,0.12645,1677.200,61309.000
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463,329.50,0.12645,1677.200,61309.000
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,beef & buffalo meat,41.3463,329.50,0.12645,1677.200,61309.000
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,beef & buffalo meat,41.3463,329.50,0.12645,1677.200,61309.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2906,I-36775,25.0,RICE CALROSE BAG,1.00,ea,20.0,Kg,FOOD - GROCERY,rice,2.5345,5.30,0.00217,1574.900,4625.600
2907,I-71940,25.0,RICE PARBOILED*,20.00,Kg,1.0,Kg,FOOD - GROCERY,rice,2.5345,5.30,0.00217,1574.900,4625.600
2908,I-72190,58.0,SAUCE MIRIN*SWEET,12.00,bottle,1.0,L,FOOD - GROCERY,sauces & paste,0.0000,6.75,0.00000,20.225,1134.925
2909,I-10481,26.0,SESAME BLK*,1.00,Kg,2.2,lb,FOOD - GROCERY,tree nuts and seeds,4.2854,12.20,0.00692,1823.300,129364.300


In [49]:
ingredients = pd.read_csv("data/preprocessed/Ingredients_List.csv")
ingredients

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
0,P-9857,5.0,Kg,1.000,1.1111,P-10112
1,P-48933,1.0,Kg,1.000,1.0000,P-10113
2,I-4626,16.0,CT,1.000,2.6667,P-10115
3,P-9755,1.0,lb,1.000,1.0000,P-10119
4,P-18746,1.0,Kg,1.000,1.0000,P-10241
...,...,...,...,...,...,...
5383,P-71658,175.0,ml,0.001,1.0000,R-73663
5384,P-73664,1.0,ea,1.000,1.0000,R-73665
5385,P-71996,1.0,PTN,1.000,1.0000,R-73666
5386,P-72051,350.0,ml,0.001,1.0000,R-73669


In [50]:
map_list = mapping["ItemId"].unique()
absent_list = []

for item in ingredients["IngredientId"].unique():
    if item not in map_list and item.startswith("I-"):
        absent_list.append(item)

# absent_list contains the IngredientIds that are not present in mapping but are present in the ingredients DataFrame
print(absent_list)

[]


In [51]:
# Converts mapping DataFrame to the Mapping.csv file
path = os.path.join(os.getcwd(), "data", "mapping", "Mapping.csv")
mapping.to_csv(path, index = False, header = True)